In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/drive/MyDrive/Assessment 2(Programming)/books_1.Best_Books_Ever.csv'
books_df = pd.read_csv(file_path)

# Display the first few rows of the dataset
books_df.head()


In [3]:
# Filtering the dataset for books in foreign languages (English)
foreign_language_books_df = books_df[books_df['language'] == 'English']

In [ ]:
import re
# Function to check if the description suggests the book is suitable for children
def is_description_suitable(description):
    if not isinstance(description, str):
        return False  # If the description is missing or not a string, we'll consider it unsuitable

    # List of keywords that might indicate unsuitability for young children
    unsuitable_keywords = ['mature', 'violence', 'sexual', 'explicit', 'horror', 'adult']

    # Basic check for the presence of these keywords
    if any(re.search(keyword, description, re.IGNORECASE) for keyword in unsuitable_keywords):
        return False
    return True

# Apply the updated function to further filter books
suitable_children_books_df = foreign_language_books_df[foreign_language_books_df['description'].apply(is_description_suitable)]

# Display the first few rows of the even more filtered dataset
suitable_children_books_df



In [ ]:
# Filter books based on a high rating (e.g. 4.0 or above)
highly_rated_children_books_df = suitable_children_books_df[(suitable_children_books_df['rating'] >= 4.0) & (suitable_children_books_df['numRatings'] >= 200000) &(suitable_children_books_df['likedPercent'] >= 95.0)]
# Display the first few rows of the dataset with highly rated books
highly_rated_children_books_df

In [40]:
# Considering books with a known price and below a certain threshold (e.g. $20)
# Convert the 'price' column to numeric values
# This step will handle any non-numeric values by converting them to NaN
highly_rated_children_books_df['price'] = pd.to_numeric(highly_rated_children_books_df['price'], errors='coerce')

# Reapply the Availability and Accessibility filter with the corrected price column
affordable_books = highly_rated_children_books_df[(highly_rated_children_books_df['price'] <= 20)]

# Display the final selection of books
affordable_books

<ipython-input-40-ad47d76a17f3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  highly_rated_children_books_df['price'] = pd.to_numeric(highly_rated_children_books_df['price'], errors='coerce')


,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,NaN,['Locus Award Nominee for Best Young Adult Boo...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,06/21/03,['Bram Stoker Award for Works for Young Reader...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38
5,19063.The_Book_Thief,The Book Thief,NaN,Markus Zusak (Goodreads Author),4.37,Librarian's note: An alternate cover edition c...,English,9780375831003,"['Historical Fiction', 'Fiction', 'Young Adult...","['Liesel Meminger', 'Hans Hubermann', 'Rudy St...",...,09/01/05,['National Jewish Book Award for Children’s an...,1834276,"['1048230', '524674', '186297', '48864', '26211']",96.0,"['Molching (Germany)', 'Germany']",https://i.gr-assets.com/images/S/compressed.ph...,1372809,14168,3.80
15,929.Memoirs_of_a_Geisha,Memoirs of a Geisha,NaN,Arthur Golden,4.12,"A literary sensation and runaway bestseller, t...",English,9781400096893,"['Fiction', 'Historical Fiction', 'Romance', '...","['Sayuri Nitta', 'Mr. Bekku', 'Ichiro Tanaka',...",...,09/23/97,[],1717312,"['712950', '615702', '295184', '66518', '26958']",95.0,"['Kyoto (Japan)', 'Japan']",https://i.gr-assets.com/images/S/compressed.ph...,862748,8993,2.76
31,8127.Anne_of_Green_Gables,Anne of Green Gables,Anne of Green Gables #1,L.M. Montgomery,4.26,As soon as Anne Shirley arrives at the snug wh...,English,9780451528827,"['Classics', 'Fiction', 'Young Adult', 'Childr...","['Marilla Cuthbert', 'Matthew Cuthbert', 'Dian...",...,10/28/08,[],727685,"['379818', '208919', '100304', '25501', '13143']",95.0,"['Avonlea, Prince Edward Island (Canada)', 'Pr...",https://i.gr-assets.com/images/S/compressed.ph...,695453,7340,6.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3556,18214414-the-silkworm,The Silkworm,Cormoran Strike #2,Robert Galbraith,4.04,Private investigator Cormoran Strike returns i...,English,9780316206877,"['Mystery', 'Fiction', 'Crime', 'Thriller', 'A...","['Cormoran Strike', 'Robin Ellacott', 'Matthew...",...,06/19/14,"['Audie Award for Mystery (2015)', 'Specsavers...",216932,"['68905', '99282', '39938', '6604', '2203']",96.0,"['London, England']",https://i.gr-assets.com/images/S/compressed.ph...,917,11,3.25
3954,35297426-then-she-was-gone,Then She Was Gone,NaN,Lisa Jewell (Goodreads Author),4.03,"THENShe was fifteen, her mother's golden girl....",English,9781501154645,"['Mystery', 'Thriller', 'Fiction', 'Mystery Th...","['Laurel Mack', 'Ellie Mack', 'Paul Mack', 'Ha...",...,NaN,['Goodreads Choice Award Nominee for Mystery &...,208605,"['66094', '93840', '39535', '6742', '2394']",96.0,[],https://i.gr-assets.com/images/S/compressed.ph...,796,9,5.97
5952,25893693-lilac-girls,Lilac Girls,Lilac Girls #1,Martha Hall Kelly (Goodreads Author),4.28,Inspired by the life of a real World War II he...,English,9781101883075,"['Historical Fiction', 'Fiction', 'World War I...","['Caroline Ferriday', 'Kasia Kuzmerick', 'Hert...",...,NaN,['Goodreads Choice Award Nominee for Historica...,213160,"['100414', '81